In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


174.7521903514862

In [3]:
casos = casos_confirmados.get_casos()
casos.shape

(332158, 18)

In [4]:
obitos = casos_confirmados.get_obitos()
obitos.shape

(6803, 11)

In [25]:
casos = casos_confirmados.get_casos()
casos['excluir'] = 'NAO'

print(f"Duplicados com a mesma idade: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
print(f"Duplicados com idade menos 1: {casos.loc[casos['hash_less'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_less'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_less'].isin(casos['hash']), 'hash_less']
print(f"Duplicados com idade mais 1: {casos.loc[casos['hash_more'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_more'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_more'].isin(casos['hash']), 'hash_more']
print(f"todos os duplicados: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
duplicados = casos.loc[casos.duplicated('hash', keep=False)]
print(f"Valores unicos: {len(duplicados.groupby('hash'))}")

Duplicados com a mesma idade: (939, 18)
Duplicados com idade menos 1: (1932, 18)
Duplicados com idade mais 1: (4, 18)
todos os duplicados: (4793, 18)
Valores unicos: 2394


In [35]:
casos.iloc[1].name

1

In [26]:
def get_better(df):
    scores = np.zeros(len(df))

    for i, serie in enumerate(df.iterrows()):
        _, row = serie

        if row['obito'] == 'SIM':
            scores[i] = 1000
        else:
            scores[i] = 1

    i = np.argmax(scores)
    return df.iloc[i].name

In [27]:
manter = []
for hash, group in duplicados.groupby('hash'):
    idx = get_better(group)
    manter.append(idx)

In [28]:
duplicados = set(duplicados.index.tolist())
duplicados = duplicados - set(manter)

casos.loc[duplicados,'excluir'] = 'SIM'
casos.loc[duplicados,'hash'] = None
casos.loc[duplicados,'hash_less'] = None
casos.loc[duplicados,'hash_more'] = None

TypeError: unhashable type: 'Index'

In [6]:
# obitos = obitos.rename(columns={'data_do_obito': 'data_cura_obito'})
# obitos['evolucao'] = 'OBITO'
# obitos = obitos[['evolucao','data_cura_obito','hash']]

# casos.loc[casos['rs'].isnull(), 'mun_resid'] = casos.loc[casos['rs'].isnull(), 'mun_resid'] + '/' + casos.loc[casos['rs'].isnull(), 'uf_resid']

# casos = pd.merge(left=casos, right=obitos, on='hash', how='left')

# casos['rs'] = None
# casos['is'] = None

casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]

ordem                   int32
ibge7                  object
nome                   object
sexo                   object
idade                   int64
mun_resid              object
mun_atend              object
rs                     object
laboratorio            object
dt_diag        datetime64[ns]
comunicacao    datetime64[ns]
is                     object
obito                  object
data_obito     datetime64[ns]
excluir                object
dtype: object

In [7]:
writer = pd.ExcelWriter("novos_casos_confirmados.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.to_excel(writer,index=None)

writer.save()
writer.close()